In [10]:
import tensorflow as tf
import os

In [15]:
#指定GPU测试
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
a = tf.constant([1.2,2.2,0.1],name = "a")
b = tf.constant ([1.0,2.0,3.0],name = "b")
result = a+b
#init_op = tf.global_variables_initializer()  这里的变量都初始化过了

config = tf.ConfigProto(allow_soft_placement=False)
#config.gpu_options.allow_growth = True#动态申请显存
session = tf.Session(config=config)
with tf.Session(config=config) as sess:
    with tf.device("/gpu:1"):
        print(sess.run(result))


In [103]:
#设计神经网络向前传播
weight1 = tf.Variable(tf.random_normal([8,5],stddev = 1,seed = 1),name = "weight1")#第一层5个节点
bias1 = tf.Variable(tf.zeros([1,5]))
weight2 = tf.Variable(tf.random_normal([5,5],stddev = 1,seed = 1))#第二层
bias2 = tf.Variable(tf.zeros([1,5]))
#定义向前传播
xs = tf.constant([[1,2,3,4,5,6,7,8]],name = "xs",dtype = tf.float32)
a = tf.matmul(xs,weight1)
a = tf.add(a,bias1)
y = tf.matmul(a,weight2)
y = tf.add(y,bias2)
all_op = tf.global_variables_initializer()
config = tf.ConfigProto(allow_soft_placement=True)
with tf.Session(config=config) as sess:
    sess.run(all_op)
    print(sess.run(y))


[[ 22.290464  -9.280622 -55.167946  50.408566  11.722612]]


In [397]:
#神经网络分类
#导入数据,和预处理
import pandas as pd
import numpy as np
from sklearn import cross_validation
name = ['name','A','B','C','D','E','target']
data = pd.read_excel(r'C:\Users\abc\Desktop\temp.xlsx',header = 0)
data.columns = name
data.pop('name')

#data = data.join(pd.get_dummies(data.target))
#data.head(5)
mapping = {1: [1,0,0,0,0],2:[0,1,0,0,0],3:[0,0,1,0,0],4:[0,0,0,1,0],5:[0,0,0,0,1]}#怎样用字典表达式写这个?
data['target'] = data['target'].map(mapping)
target = data.pop('target')
target = list(target.values)
target = np.array(target)
X_train, X_test, y_train, y_test = cross_validation.train_test_split(data,target, test_size=0.3, random_state=0)
#数据预处理
#热编码处理标签
classes = 5
#使用placehoder机制（定义一个位置）
#这个模型有个致命缺陷，就是多分类问题的激活函数，二分类才能为sigmoid
weight1 = tf.Variable(tf.random_normal([5,5],stddev = 1,seed = 1),name = "weight1")#第一层5个节点
bias1 = tf.Variable(tf.zeros([1,5]),dtype = tf.float32)
weight2 = tf.Variable(tf.random_normal([5,5],stddev = 1,seed = 1))#第二层
bias2 = tf.Variable(tf.zeros([1,5]),dtype = tf.float32)
#定义向前传播
xs = tf.placeholder(tf.float32,shape =(None,5),name = 'input')
y_ = tf.placeholder(tf.float32,shape =(None,5),name = 'input')
a = tf.matmul(xs,weight1)
a = tf.add(a,bias1)
a = tf.nn.relu(a)
y = tf.matmul(a,weight2)
y = tf.add(y,bias2)
y = tf.nn.softmax(y)
#GPU参数
config = tf.ConfigProto(allow_soft_placement=True)
#y_ = tf.one_hot(y_,classes)
#定义反向传播
cross_entropy = -tf.reduce_mean(y_*tf.log(tf.clip_by_value(y,1e-10,1.0)))#定义
learning_rate = 0.001
#定义反向传播参数
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)
#运行train_step就可以启动优化器

step = 100#训练轮次
batch_size = 10
tf.Session(config=config).run(all_op)
all_op = tf.global_variables_initializer()
with tf.Session(config=config) as sess:
    sess.run(all_op)
    for i in range(step):
        start = i*batch_size
        end = min((i+1)*batch_size,len(X_train))
        sess.run(train_step,feed_dict = {xs:X_train[start:end],y_:y_train[start:end]})
        if i%50==0:
            total_cross_entropy = sess.run(cross_entropy,feed_dict ={xs:X_train,y_:y_train})#每隔一段时间输出模型总误差#是不是有叠加entropy？
            print(i,total_cross_entropy)  
        if end == len(X_train):
            break
    print(sess.run(weight1))
    print(sess.run(weight2))
with tf.Session(config=config) as sess:
    sess.run(all_op)
    temp = sess.run(y,feed_dict = {xs:X_test})
temp1 = []
for i in range(temp.shape[0]):
    max_ = np.argmax(temp[i]) 
    i = np.zeros([5])
    i[max_] = 1
    temp1.append(i)
temp1-y_test

0 1.302495
[[-0.8113182   1.4845958   0.06614839 -2.4427042   0.09720233]
 [ 0.5912243   0.5927074  -2.1223874  -0.72289723 -0.05756911]
 [ 0.6435448  -0.26464468  1.8576094   0.5678417  -0.38462368]
 [-1.4853433   1.2615813  -0.02447971 -0.2646297   1.5312678 ]
 [-1.7429771  -0.4382256  -0.5656303   0.32066926  1.1324259 ]]
[[-0.8113182   1.4845988   0.06532937 -2.4427042   0.0992484 ]
 [ 0.59397745  0.5906697  -2.1229296  -0.7221895  -0.05757786]
 [ 0.6437825  -0.2644888   1.8566332   0.5681105  -0.38317776]
 [-1.4853433   1.2617711  -0.02530608 -0.2646297   1.5328138 ]
 [-1.7421483  -0.43849617 -0.56601     0.32169715  1.1315775 ]]


In [14]:
#回归问题：
import tensorflow as tf
import random
import numpy as np

#先定义获取layer的函数
def get_weight(shape, lambda_,number=None):
    bias = tf.Variable(tf.zeros(shape=[1,shape[1]]),dtype = tf.float32)
    weight = tf.Variable(tf.random_normal(shape),name = number,dtype = tf.float32)
    tf.add_to_collection('losses',tf.contrib.layers.l2_regularizer(lambda_)(weight))#wi为变量的正则表达式，并且加入集合
    return weight,bias
#预处理和设置数据输入
dataset_size = 128
feature_num = 2
batch_size = 10
step = dataset_size%batch_size
#制作数据集和数据容器

x = np.random.rand(dataset_size,feature_num)
x4 = x[:,0:2]#narray是不可以改变值的
#x__.dtype = np.float32
y = np.array([[x1+2*x2+np.random.rand()/10.0-0.05]for x1,x2 in x4],dtype=np.float32)
x4.dtype = np.float32
x3 = x4[:,0:2]
y.reshape(-1,1)
result_num = 1
x_ = tf.placeholder(tf.float32,shape = (None,2))
print (x.dtype)
y_ = tf.placeholder(tf.float32,shape = (None,1))

#定义每层节点数，包括输入层输出层
layer_dimension = [2,10,10,10,1]
#隐藏层数目
layer_num = len(layer_dimension)
#上一层
for_layer = x
for i in range(1,layer_num):
    out_dimension = layer_dimension[i]
    #生成当前层权重
    weight,bias = get_weight([layer_dimension[i-1],out_dimension],0.01)
    #激活函数激活
    for_layer = tf.nn.relu(tf.matmul(tf.cast(for_layer,dtype = tf.float32),weight)+bias)
#计算损失
mse_loss = tf.reduce_mean(tf.square(y_-for_layer))
#加入损失集合
tf.add_to_collection('losses',mse_loss)
loss_total = tf.add_n(tf.get_collection('losses'))#把损失集合全部加起来
#设置训练：
learning_rate = 0.001
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_total)

#GPU参数
#config = tf.ConfigProto(allow_soft_placement=True)
step = 12
all_op = tf.global_variables_initializer()
config = tf.ConfigProto(allow_soft_placement=True)
tf.Session(config=config).run(all_op)
with tf.Session(config=config) as sess:
    for i in range(step):
        start = 0*batch_size
        end = start+batch_size
        sess.run(train_step,feed_dict={x_:x3[start:end],y_:y[start:end]})
        sess.run(train_step,feed_dict={x_:x3,y_:y})#有验证集这里换成验证集
        


array([[2.3539972 ],
       [1.8623776 ],
       [0.9386359 ],
       [0.9306019 ],
       [1.7364984 ],
       [1.9444269 ],
       [1.1462723 ],
       [1.6304936 ],
       [2.3553066 ],
       [0.4577079 ],
       [1.1387763 ],
       [0.71998715],
       [1.8983144 ],
       [1.9846485 ],
       [2.6205149 ],
       [0.42431557],
       [1.2465173 ],
       [1.4135623 ],
       [1.2602592 ],
       [0.9500469 ],
       [1.4910152 ],
       [2.241445  ],
       [1.135291  ],
       [2.8446696 ],
       [1.4682897 ],
       [0.8729176 ],
       [2.499122  ],
       [1.4275925 ],
       [2.894475  ],
       [2.3602004 ],
       [2.5505965 ],
       [1.1910179 ],
       [1.7240503 ],
       [2.108063  ],
       [2.0506706 ],
       [2.0828876 ],
       [1.8757933 ],
       [2.3503175 ],
       [1.413803  ],
       [0.71637535],
       [1.9603761 ],
       [1.3804647 ],
       [0.3603826 ],
       [1.1762816 ],
       [1.736786  ],
       [0.46622884],
       [1.3679407 ],
       [1.180

In [3]:
#运用所有优化方法
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
#minst运用所有方法包括正则化，移动平滑
input_size = 784
output_size = 10
#神经网络参数 （一个隐藏层）
layer_size = 500
batch_size = 100#随机梯度下降法
learning_rate_origin = 0.8 #初始学习率
learning_rate_decay = 0.99#衰减率
lambda_ = 0.0001#正则化lambda
train_srep = 30000#训练轮次
moving_average_decay = 0.99
#辅助函数，给定参数计算向前传播结果
#Relu的三层全连接
def inference(input_tensor,avg_class,weights1,biases1,weights2,biases2):
    if avg_class ==None:#未提供滑动平均
        layer1 = tf.nn.relu(tf.matmul(input_tensor,weights1)+biases1)
        #计算损失函数时使用softmax先激活
        return tf.matmul(layer1,weights2)+biases2
    else :#先计算滑动平均再向前传播
        layer1 = tf.nn.relu(tf.matmul(input_tensor,avg_class.average(weights1))+avg_class.average(biases1))
        return tf.matmul(layer1,avg_class.average(weights2))+avg_class.average(biases2)
#模型训练过程
def train(mnist):
    x = tf.placeholder(tf.float32,[None,input_size],name = 'input')
    y_ = tf.placeholder(tf.float32,[None,10],name = 'output')
    weights1 = tf.Variable(tf.truncated_normal([input_size,layer_size],stddev = 0.1))#784 * 500
    biases1 =  tf.Variable(tf.constant(0.1,shape = [layer_size]))
    #输出层
    weights2 = tf.Variable(tf.truncated_normal([layer_size,output_size],stddev = 0.1))
    biases2 =  tf.Variable(tf.constant(0.1,shape = [output_size]))
    #不使用滑动平均
    #y = inference(x,None,weights1,biases1,weights2,biases2)
    global_step = tf.Variable(0,trainable=False)
    #初始化平滑
    variable_averages = tf.train.ExponentialMovingAverage(moving_average_decay,global_step)
    variable_op = variable_averages.apply(tf.trainable_variables())
    average_y = inference(x,variable_averages,weights1,biases1,weights2,biases2)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=tf.argmax(y_, 1), logits=average_y)
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    regularizer = tf.contrib.layers.l2_regularizer(lambda_)
    regularization = regularizer(weights1)+regularizer(weights2)
    loss = cross_entropy_mean+regularization 
    #设置指数衰减率
    #global_step#当前迭代轮次
    learning_rate = tf.train.exponential_decay(learning_rate_origin,global_step,mnist.train.num_examples/batch_size,learning_rate_decay)
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step = global_step)
#同时完成反向传播的更新参数和滑动平均的更新参数
    train_op = tf.group(train_step,variable_op)
#预测是否正确
    correct_prediction = tf.equal(tf.argmax(average_y,1),tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    all_op = tf.global_variables_initializer()
    config = tf.ConfigProto(allow_soft_placement=True)
    
    with tf.Session(config=config) as sess:
        sess.run(all_op)
        validate_feed = {x:mnist.validation.images,y_:mnist.validation.labels}
        test_feed = {x:mnist.test.images,y_:mnist.test.labels}
        for i in range(train_srep):
            if i % 1000 == 0:
                sess.run(all_op)
                print(str(i),':',sess.run(accuracy,feed_dict = validate_feed))
        xs,ys = mnist.train.next_batch(batch_size)
        sess.run(train_op,feed_dict = {x:xs,y_:ys})
        test_acc = sess.run(accuracy,feed_dict=test_feed)
        print('final_accuracy:',test_acc)
#主程序
def main(argv = None):
    train(mnist)
if __name__ =='__main__':
    tf.app.run()





                           

0 : 0.1162
1000 : 0.094
2000 : 0.0634
3000 : 0.0696
4000 : 0.123
5000 : 0.1276
6000 : 0.102
7000 : 0.0966
8000 : 0.1218
9000 : 0.0484
10000 : 0.163
11000 : 0.104
12000 : 0.0878
13000 : 0.0704
14000 : 0.0876
15000 : 0.1364
16000 : 0.0692
17000 : 0.0802
18000 : 0.1036
19000 : 0.1094
20000 : 0.1172
21000 : 0.1128
22000 : 0.0778
23000 : 0.1086
24000 : 0.085
25000 : 0.0768
26000 : 0.115
27000 : 0.1056
28000 : 0.1084
29000 : 0.1
final_accuracy: 0.109


SystemExit: 

D:\vs2017\Anaconda3_64\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
#使用tensorflow查询机器上是否存在可用的gpu设备
def is_gpu_available(cuda_only=True):
  """
  code from https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/platform/test.py
  Returns whether TensorFlow can access a GPU.
  Args:
    cuda_only: limit the search to CUDA gpus.
  Returns:
    True iff a gpu device of the requested kind is available.
  """
  from tensorflow.python.client import device_lib as _device_lib
 
  if cuda_only:
    return any((x.device_type == 'GPU')
               for x in _device_lib.list_local_devices())
  else:
    return any((x.device_type == 'GPU' or x.device_type == 'SYCL')
               for x in _device_lib.list_local_devices())
#使用tensorflow获取可用的gpu设备编号

def get_available_gpus():
    """
    code from http://stackoverflow.com/questions/38559755/how-to-get-current-available-gpus-in-tensorflow
    """
    from tensorflow.python.client import device_lib as _device_lib
    local_device_protos = _device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']


In [2]:
get_available_gpus()

[]

In [ ]:
import tensorflow as tf
with tf.Session() as ses:
    with tf.device("/gpu:1"):
        matrix1=tf.constant([[3.,3.]])
        matrix2=tf.constant([[2.],[2.]])
        product=tf.matmul(matrix1,matrix2)
        ses.run(product)
